In [96]:
# %load GoodsClassifier.py

import os
# -*- coding: utf-8 -*-
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from annoy import AnnoyIndex

In [97]:
features_path = "data/goods_features/"
index_path = features_path + "resnet_256.idx"
pretrained_features_path = features_path + "finetuned_weights.h5"

img_height = 224
img_width = 224
batch_size = 64

nb_train_samples = 611889
nb_validation_samples = 67799

nb_classes = 844

dataset_path = "classifieds/"
train_dir = dataset_path + 'train'
validation_dir = dataset_path + 'validation'

In [98]:
def insert_to_annoy(images, directory):
    global counter_global
    for img in images:
        img = preprocess_input(img_to_array(load_img(dataset_path+direct+"/"+img,target_size=(224,224))))
        img = np.expand_dims(img,axis=0)
        prediction = np.reshape(model.predict(img)[0],vector_size)
        annoy_global.add_item(counter_global,prediction)
        counter_global += 1

In [99]:
# модель = ResNet50 без голови з одним dense шаром для класифікації об'єктів на nb_classes
def get_model(nb_classes=844, fine_tune=False, weights_path=None, layers_unfreeze=3, lr=1e-4, epochs=10):
    model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    
    if not fine_tune:        
        optimizer = Adam(lr=lr)
    else:
        optimizer = SGD(lr=lr, momentum=0.9, nesterov=True)
    for layer in model.layers[:-layers_unfreeze]:
        layer.trainable = False
        
    flat = Flatten()(model.output)  
    # можна додати кілька dense шарів:
    d = Dense(2048, activation='relu')(flat)
    d = Dropout(0.2)(d)
    d = Dense(nb_classes, activation='softmax')(d)
    model = Model(inputs=model.input, outputs=d)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    
    if weights_path:
        model.load_weights(weights_path)
        
    model.summary()
    return model

In [100]:
def train_model(nb_classes, layers_unfreeze=3, lr=1e-4, fine_tune=False, weights_path=None, epochs=10):

    model = get_model(nb_classes, fine_tune=fine_tune, weights_path=weights_path,
                      layers_unfreeze=layers_unfreeze, lr=lr)
#     train_gen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)
        
    validation_gen = ImageDataGenerator(rescale=1. / 255)
    train_gen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_gen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = validation_gen.flow_from_directory(
        validation_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    model.fit_generator(
        generator=train_generator,
        validation_data=validation_generator,
        validation_steps=1000,
        steps_per_epoch=1000,
        nb_epoch=epochs,
        shuffle=True,
        callbacks=[ModelCheckpoint(pretrained_features_path, save_best_only=True, monitor='val_loss')])

    model.save_weights(pretrained_features_path)
    
    return model

In [103]:
def startTraining(nb_classes):
#     train_model(nb_classes, weights_path=pretrained_features_path, layers_unfreeze=3, lr=1e-4, epochs=10)
#     train_model(nb_classes, weights_path=pretrained_features_path, layers_unfreeze=8, lr=1e-5, epochs=20)
    model = train_model(nb_classes, fine_tune=True, weights_path=pretrained_features_path, 
                        layers_unfreeze=12, lr=1e-5, epochs=5)
    
    return model    

In [104]:
model = startTraining(nb_classes)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_26 (InputLayer)            (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_26[0][0]                   
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1226 (Activation)     (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

Found 611889 images belonging to 844 classes.
Found 67799 images belonging to 844 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(shuffle=True, generator=<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=1000, epochs=5, callbacks=[<keras.ca..., validation_steps=1000)`


Epoch 1/5
1000/1000 [==============================] - 1630s - loss: 2.0624 - acc: 0.5248 - val_loss: 2.2750 - val_acc: 0.4873
Epoch 2/5
  49/1000 [>.............................] - ETA: 424s - loss: 2.0660 - acc: 0.5258

KeyboardInterrupt: 

In [ ]:
vector_size = 2048
n_trees = 256

In [ ]:
def predict_n_neighbours(targetImagePath, topn=5):
    annoy = AnnoyIndex(vector_size, metric='angular')
    annoy.load(index_path)
    